In [ ]:
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path_temprature = r"Data" + "\\"
path_fig = r"Figure\Scatter" + "\\"
path_df = r"Table" + "\\"

sns.set_theme(style="white", font="Microsoft YaHei")

font1 = {"family": "Microsoft YaHei", "weight": "bold", "size": 18}

In [ ]:
model = "XGBoost"

N1 = 50  # Number of initial features
N2 = 20  # Number of final features

In [ ]:
df = []
file_list = glob.glob(path_temprature + "*.csv")
file_list.sort()


for file in file_list:

    df1 = pd.read_csv(file, parse_dates=["Date"])

    df1.dropna(subset=["Temperature"], inplace=True)

    df.append(df1)


df = pd.concat(df, axis=0, ignore_index=True)

features = df.columns.drop(["River", "Station", "Date", "Temperature"])

features = [f for f in features if "basin_" not in f]

df

# Cross validation

In [ ]:
def initial_feature_selection(X_train, y_train, X_test):
    # Initial feature selection by feature importance ranking
    model = XGBRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    feature_importances = model.feature_importances_
    feature_index = np.argsort(feature_importances)[::-1][:N1]
    X_train1 = X_train.iloc[:, feature_index]
    X_test1 = X_test.iloc[:, feature_index]

    return X_train1, X_test1


def rfe_feature_selection(X_train, y_train, X_test):
    # Final feature selection by RFE
    xgb = XGBRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    rfe = RFE(estimator=xgb, n_features_to_select=N2, step=1)

    rfe.fit(X_train, y_train)
    ranking = rfe.ranking_
    ranking = pd.DataFrame(
        ranking, index=X_train.columns, columns=["Rank"]
    ).sort_values(by="Rank", ascending=True)
    ranking.index.name = "Feature"
    features_selected = ranking[ranking["Rank"] == 1]

    X_train2 = X_train[features_selected.index]
    X_test2 = X_test[features_selected.index]

    return X_train2, X_test2, features_selected.index


def hyperparameter_tuning(X_train, y_train):
    param_grid = {
        "max_depth": [3, 4],
        "learning_rate": [0.005, 0.01, 0.015],
        "n_estimators": [300, 400],
        "subsample": [0.6, 0.7, 0.8],
        "colsample_bytree": [0.5, 0.7, 1],
    }

    # Using RandomizedSearchCV to find the best hyperparameter combination
    xgb = XGBRegressor(random_state=42, n_jobs=-1)
    xgb_random = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_grid,
        n_iter=100,
        cv=5,
        verbose=2,
        random_state=42,
        n_jobs=-1,
    )
    xgb_random.fit(X_train, y_train)

    # Get the best model and its hyperparameter combination
    model = xgb_random.best_estimator_

    return model


def scatter_plot(y_test, y_pred, river, station):
    fig, ax = plt.subplots()

    ax.set_aspect("equal")
    ax.scatter(y_test, y_pred, s=2)

    min_value = min(min(y_test), min(y_pred))
    max_value = max(max(y_test), max(y_pred))
    ax.plot(
        [min_value, max_value],
        [min_value, max_value],
        color="grey",
        linewidth=1,
        linestyle="--",
    )

    ax.set_xlabel("In-situ Temperature (°C)")
    ax.set_ylabel("Predicted Temperature (°C)")
    ax.set_title(f"Model validation for excluding {river}")

    fig.tight_layout()
    plt.savefig(path_fig + f"Scatter__{river}__{station}.png", dpi=300)
    plt.close()


def build_model(river, station):
    print(river, station)
    df_train = df[df["Station"] != station]
    df_test = df[df["Station"] == station]

    X_train = df_train[features]
    y_train = df_train["Temperature"]
    X_test = df_test[features]
    y_test = df_test["Temperature"]

    X_train1, X_test1 = initial_feature_selection(X_train, y_train, X_test)
    X_train2, X_test2, feature_names = rfe_feature_selection(X_train1, y_train, X_test1)
    model = hyperparameter_tuning(X_train2, y_train)

    y_train_pred = model.predict(X_train2).clip(min=0)
    y_test_pred = model.predict(X_test2).clip(min=0)
    scatter_plot(y_test, y_test_pred, river, station)
    river_station = river + "__" + station
    m1 = calc_metric(y_train, y_train_pred, 3, river_station)
    m1.index = [f"{idx}_train" for idx in m1.index]
    m2 = calc_metric(y_test, y_test_pred, 3, river_station)
    # m2.index = [f"{idx}_test" for idx in m2.index]
    m = pd.concat([m1, m2], axis=0)

    if m["R2_train"] - m["R2"] > 0.1:
        m["Overfitting"] = True
        print("Possible overfitting!!")
    else:
        m["Overfitting"] = False

    return m, y_test_pred


def calc_metric(y_true, y_pred, decimal_digit=3, name=None):
    from sklearn.metrics import r2_score, mean_absolute_error

    # Align y_true and y_pred and drop NaN values
    combined = pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).dropna()
    y_true = combined["y_true"].values
    y_pred = combined["y_pred"].values

    R = np.corrcoef(y_true, y_pred)[0, 1]
    R2 = r2_score(y_true, y_pred)
    MAE = mean_absolute_error(y_true, y_pred)
    try:
        from sklearn.metrics import mean_squared_error

        RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    except ImportError:
        from sklearn.metrics import root_mean_squared_error

        RMSE = root_mean_squared_error(y_true, y_pred)

    NRMSE = RMSE / (np.max(y_true) - np.min(y_true)) * 100
    DDR = np.mean(
        np.abs((y_pred - y_true) / np.mean(y_true))
    )  # Developed Discrepancy Ratio (custom calculation)
    IQR = np.percentile(y_true, 75) - np.percentile(y_true, 25)  # Inter-Quartile Range
    RPIQ = IQR / RMSE  # Ratio of Performance to Inter-Quartile Range

    # Calculate KGE
    alpha = np.std(y_pred) / np.std(y_true)
    beta = np.mean(y_pred) / np.mean(y_true)
    KGE = 1 - np.sqrt((R - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

    metrics = {
        "R": R,
        "R2": R2,
        "MAE": MAE,
        "RMSE": RMSE,
        "NRMSE (%)": NRMSE,
        "DDR": DDR,
        "RPIQ": RPIQ,
        "KGE": KGE,
    }
    # Convert to series
    metrics = pd.Series(metrics).round(decimal_digit)
    if name:
        metrics.name = name

    return metrics

In [ ]:
result = []
for station in df["Station"].unique()[:]:
    row = df[df["Station"] == station].iloc[0]
    river = row["River"]
    metrics, y_pred = build_model(river, station)
    result.append(metrics)

result = pd.concat(result, axis=1).T
result.sort_values("R2", inplace=True, ascending=False)

In [ ]:
result

In [ ]:
for index, row in result.iterrows():
    river, station = index.split("__")
    result.loc[index, "River"] = river
    result.loc[index, "Station"] = station

result.reset_index(drop=True, inplace=True)
cols = result.columns.tolist()
cols = cols[-2:] + cols[:-2]
result = result[cols]
result

In [ ]:
r2 = result["R2"]
print("Max R2:", r2.max(), "Min R2:", r2.min())
print("Median R2:", r2.median())

kge = result["KGE"]
print("Max KGE:", kge.max(), "Min KGE:", kge.min())
print("Median KGE:", kge.median())

nrmse = result["NRMSE (%)"]
print("Max NRMSE:", nrmse.max(), "Min NRMSE:", nrmse.min())
print("Median NRMSE:", nrmse.median())

In [ ]:
result.to_csv(path_df + f"Cross_validation Metrics_{model}.csv", index=False)